In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-11-01'
final_date = '2022-11-30'
query_error_year = 2022
query_error_month = 11

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\11. NOVIEMBRE\DATA\ERROR NOVIEMBRE 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(111959, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1976\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
101103,101103,ee8c6855-c796-4db9-81f7-8534f353aa5b,Takos De kanasta Rios,LKHX3J5MR,2022-11-01,90.08,ERROR,R8,2022-11-01,11,2022,NEW,ERROR_USUAL,ee8c6855-c796-4db9-81f7-8534f353aa5b,2022-11-01,1,Actual
99677,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual
99659,99659,d48beed5-51e0-43d7-84c8-57754b61f4a2,SUPERTICK,KQ32L6S5Z,2022-10-28,127.49,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,d48beed5-51e0-43d7-84c8-57754b61f4a2,2022-11-01,1,Actual
99657,99657,5665e03f-407f-49a8-bf85-ff4e28bf862d,City Cell San Nicolas,24T7JZ4J9,2022-10-28,400.62,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,5665e03f-407f-49a8-bf85-ff4e28bf862d,2022-11-01,1,Actual
99656,99656,36c02f4b-22c4-4357-8ae8-bd9071793a92,Wet Cleaning,2ZCQC564Q,2022-10-28,480.16,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,36c02f4b-22c4-4357-8ae8-bd9071793a92,2022-11-01,1,Actual


In [9]:
query_error_current_month.shape

(11341, 17)

In [10]:
query_error_current_month['month'].unique()

array([11], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1976\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
46519,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN
82019,82019,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,LNCNLVR9V,2022-08-11,0.01,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN
97190,97190,c1ff0f5a-9a4c-48c7-8621-df7a9509c674,Abarrotes Castillo,ZXS9G8C5K,2022-10-25,89.11,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-10-27,10,2022,NEW,ERROR_USUAL,c1ff0f5a-9a4c-48c7-8621-df7a9509c674,2022-10-27,27,NaN
97191,97191,7b3ed5f1-0fd0-44f8-9240-eec35a6d780f,Foto Cafe,G9S4LHNHL,2022-10-25,364.14,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-10-27,10,2022,NEW,ERROR_USUAL,7b3ed5f1-0fd0-44f8-9240-eec35a6d780f,2022-10-27,27,NaN


In [15]:
query_new_last_month.shape

(4, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\11. NOVEMBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\11. NOVIEMBRE\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [23]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\10. OCTOBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\10. OCTUBRE\QUERIES\*.csv', recursive=False)

In [24]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [25]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [28]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\09. SEPTEMBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\09. SEPTIEMBRE\QUERIES\*.csv', recursive=False)

In [29]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [30]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [31]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
11693,DAILY,21a9fde8-6e3c-4eca-9379-c364ccfbb71f,Helados Thrifty,jlpartida@hotmail.com,22236J8TB,2022-09-01,292.26,"Banco Santander, S.A.",2022-09-01 10:32:24,JOSE LUIS PARTIDA BOBADILLA,'014560570031216777,2022-09-01 08:06:21,SANTANDER,"JOSE LUIS PARTIDA BOBADILLA,,292.26,0145605700...",SANT-1 2022-09-01.csv,2022-09-01,TRAD
82470,DAILY,ed9cd493-d59c-4bc0-9897-1e2659c4b7ee,Dulces y mas el jordan,clanton14pecaslatozo@gmail.com,2228TT7PR,2022-09-01,565.37,"HSBC México, S.A.",2022-09-01 09:23:59,David Jordan Flores Villeda,'021180065432631363,2022-09-01 08:18:00,SANTANDER,"David Jordan Flores Villeda,,565.37,0211800654...",SANT-1 2022-09-01.csv,2022-09-01,TRAD
55197,DAILY,9e649a79-8e0d-4827-a66f-28c2a9872a8b,Cremeria lety,letyvillanueva1118@gmail.com,222C6KQ6L,2022-09-01,125.52,"BBVA Bancomer, S.A.",2022-09-01 10:48:20,Marcos Castro Villanueva,'012180015217106920,2022-09-01 08:13:07,SANTANDER,"Marcos Castro Villanueva,,125.52,0121800152171...",SANT-1 2022-09-01.csv,2022-09-01,TRAD
4005,INSTANT,c511de59-24c4-4e03-a547-73f35913a9d6,Farmacia VCR,tania.espinoza1@gmail.com,222DS8V5W,2022-09-01,57.49,"Opciones Empresariales del Noreste, S.A. de C....",2022-09-01 14:14:51,Tania Marlene Espinoza Garcia,'659802020000031459,2022-09-01 14:05:46,SANTANDER,"Tania Marlene Espinoza Garcia,,57.49,659802020...",FP-ALL 2022-09-01.csv,2022-09-01,FP
25124,DAILY,480ad868-c4b9-43ea-8eec-9972af6a5362,Be Lo Style,bgaliciag@hotmail.com,222R5XR8M,2022-09-01,"9,774.05","Banco Nacional de México, S.A.",2022-09-01 10:30:48,Berenice Galicia Garduño,'002180901386585884,2022-09-01 08:08:23,SANTANDER,"Berenice Galicia Garduño,,9774.05,002180901386...",SANT-1 2022-09-01.csv,2022-09-01,TRAD


In [32]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [33]:
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
11693,DAILY,21a9fde8-6e3c-4eca-9379-c364ccfbb71f,Helados Thrifty,jlpartida@hotmail.com,22236J8TB,2022-09-01,292.26,"Banco Santander, S.A.",2022-09-01 10:32:24,JOSE LUIS PARTIDA BOBADILLA,'014560570031216777,2022-09-01 08:06:21,SANTANDER,"JOSE LUIS PARTIDA BOBADILLA,,292.26,0145605700...",SANT-1 2022-09-01.csv,2022-09-01,TRAD,Pago del día
82470,DAILY,ed9cd493-d59c-4bc0-9897-1e2659c4b7ee,Dulces y mas el jordan,clanton14pecaslatozo@gmail.com,2228TT7PR,2022-09-01,565.37,"HSBC México, S.A.",2022-09-01 09:23:59,David Jordan Flores Villeda,'021180065432631363,2022-09-01 08:18:00,SANTANDER,"David Jordan Flores Villeda,,565.37,0211800654...",SANT-1 2022-09-01.csv,2022-09-01,TRAD,Pago del día
55197,DAILY,9e649a79-8e0d-4827-a66f-28c2a9872a8b,Cremeria lety,letyvillanueva1118@gmail.com,222C6KQ6L,2022-09-01,125.52,"BBVA Bancomer, S.A.",2022-09-01 10:48:20,Marcos Castro Villanueva,'012180015217106920,2022-09-01 08:13:07,SANTANDER,"Marcos Castro Villanueva,,125.52,0121800152171...",SANT-1 2022-09-01.csv,2022-09-01,TRAD,Pago del día
4005,INSTANT,c511de59-24c4-4e03-a547-73f35913a9d6,Farmacia VCR,tania.espinoza1@gmail.com,222DS8V5W,2022-09-01,57.49,"Opciones Empresariales del Noreste, S.A. de C....",2022-09-01 14:14:51,Tania Marlene Espinoza Garcia,'659802020000031459,2022-09-01 14:05:46,SANTANDER,"Tania Marlene Espinoza Garcia,,57.49,659802020...",FP-ALL 2022-09-01.csv,2022-09-01,FP,Pago del día
25124,DAILY,480ad868-c4b9-43ea-8eec-9972af6a5362,Be Lo Style,bgaliciag@hotmail.com,222R5XR8M,2022-09-01,"9,774.05","Banco Nacional de México, S.A.",2022-09-01 10:30:48,Berenice Galicia Garduño,'002180901386585884,2022-09-01 08:08:23,SANTANDER,"Berenice Galicia Garduño,,9774.05,002180901386...",SANT-1 2022-09-01.csv,2022-09-01,TRAD,Pago del día


In [34]:
allfiles.shape

(8498160, 18)

# CRUCE 1 - MES ACTUAL

In [35]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,21a9fde8-6e3c-4eca-9379-c364ccfbb71f,Helados Thrifty,jlpartida@hotmail.com,22236J8TB,2022-09-01,292.26,"Banco Santander, S.A.",2022-09-01 10:32:24,JOSE LUIS PARTIDA BOBADILLA,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,ed9cd493-d59c-4bc0-9897-1e2659c4b7ee,Dulces y mas el jordan,clanton14pecaslatozo@gmail.com,2228TT7PR,2022-09-01,565.37,"HSBC México, S.A.",2022-09-01 09:23:59,David Jordan Flores Villeda,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,9e649a79-8e0d-4827-a66f-28c2a9872a8b,Cremeria lety,letyvillanueva1118@gmail.com,222C6KQ6L,2022-09-01,125.52,"BBVA Bancomer, S.A.",2022-09-01 10:48:20,Marcos Castro Villanueva,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,INSTANT,c511de59-24c4-4e03-a547-73f35913a9d6,Farmacia VCR,tania.espinoza1@gmail.com,222DS8V5W,2022-09-01,57.49,"Opciones Empresariales del Noreste, S.A. de C....",2022-09-01 14:14:51,Tania Marlene Espinoza Garcia,...,FP,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,480ad868-c4b9-43ea-8eec-9972af6a5362,Be Lo Style,bgaliciag@hotmail.com,222R5XR8M,2022-09-01,"9,774.05","Banco Nacional de México, S.A.",2022-09-01 10:30:48,Berenice Galicia Garduño,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [36]:
cruce_1.shape

(8498160, 26)

In [37]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
136,DAILY,7772f9bd-1d8b-4aaf-a042-4682d25a16e1,Abarrotes el puente,materialderecho101@gmail.com,238S3VFPZ,2022-09-01,"9,582.40",AZTECA,2022-09-01 10:45:47,Gustavo Omar Pastén Rodríguez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1393,DAILY,67687617-4f7f-4c34-bdfd-9c36247d83d2,Los De A diez,deyri_puma@hotmail.com,2GYYYLLK5,2022-09-01,4.00,"Banco Compartamos, S.A.",2022-09-01 10:33:32,Denisse Martinez Bermudez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4962,DAILY,04d46dcd-745a-43fe-8fcb-ad9b0f71f2d8,Beijing,yongxingyou869@gmail.com,3KTXF9D7C,2022-09-01,1.90,"Banco Nacional de México, S.A.",2022-09-01 10:29:53,YONGXING YOU,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
5550,DAILY,3a906075-15ba-4ca4-90b6-b042466e83f3,SWING PASTA 2,pastaswing2@gmail.com,3QXCDWLX6,2022-09-01,"4,276.60","BBVA Bancomer, S.A.",2022-09-01 10:29:38,GUADALUPE MONTSERRAT RAMIREZ POLANCO,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
5654,DAILY,caa7fc6f-3d20-4297-9eb8-bea561aa1727,Ferretera El Marino,manuelcas_@hotmail.com,3RPVBMDR3,2022-09-01,573.51,"Banco Azteca, S.A.",2022-09-01 10:32:17,Manuel De Jesus Castellanos Aguilar,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [38]:
duplicates_cruce_1_previo.shape

(56658, 26)

In [39]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1976\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
3498049,DAILY,75f6e4f2-e325-4ff3-aca7-5ce650363a0c,Vestua,carolina.rosen@vestua.com,B9L79T62V,2022-10-09,"2,433.94","Banco Nacional de México, S.A.",2022-10-10 06:24:39,Vestua Sa De Cv,...,TRAD,Pago del día,"2,433.94",2022-11-29,2022-11-29,11.00,ERROR_RES,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,ERROR
3614647,DAILY,75f6e4f2-e325-4ff3-aca7-5ce650363a0c,Vestua,carolina.rosen@vestua.com,N49RBB9WQ,2022-10-10,"2,788.48","Banco Nacional de México, S.A.",2022-10-10 09:11:57,Vestua Sa De Cv,...,TRAD,Pago del día,"2,788.48",2022-11-29,2022-11-29,11.00,ERROR_RES,DEVOLUCION(02): Cuenta bloqueada 002180...,ERROR_USUAL,ERROR
4536459,DAILY,0980edb2-ac50-4f66-a92d-ebbd5627f5c3,BODY TONER S DE RL DE CV,bodytonerzdi@outlook.com,J55S99QVR,2022-10-20,"13,462.07","Banco Santander, S.A.",2022-10-20 09:18:03,BODY TONER S DE RL DE CV,...,TRAD,Pago del día,"13,462.07",2022-11-08,2022-11-08,11.00,ERROR,Error request by Backoffice,ERROR_USUAL,ERROR
4882824,DAILY,75f6e4f2-e325-4ff3-aca7-5ce650363a0c,Vestua,carolina.rosen@vestua.com,Y4K8T7H2C,2022-10-23,373.72,"Banco Nacional de México, S.A.",2022-10-24 06:26:24,Vestua Sa De Cv,...,TRAD,Pago del día,373.72,2022-11-29,2022-11-29,11.00,ERROR_RES,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,ERROR
4928056,DAILY,75f6e4f2-e325-4ff3-aca7-5ce650363a0c,Vestua,carolina.rosen@vestua.com,K73DQPM6Y,2022-10-24,440.79,"Banco Nacional de México, S.A.",2022-10-24 11:57:41,Vestua Sa De Cv,...,TRAD,Pago del día,440.79,2022-11-29,2022-11-29,11.00,ERROR_RES,DEVOLUCION(02): Cuenta bloqueada 002180...,ERROR_USUAL,ERROR


In [40]:
cruce_1_final.shape

(6428, 26)

In [41]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [42]:
duplicates_cruce_1_final.shape

(0, 26)

# CRUCE 2 MES ACTUAL

In [43]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,101103,ee8c6855-c796-4db9-81f7-8534f353aa5b,Takos De kanasta Rios,LKHX3J5MR,2022-11-01,90.08,ERROR,R8,2022-11-01,11,2022,NEW,ERROR_USUAL,ee8c6855-c796-4db9-81f7-8534f353aa5b,2022-11-01,1,Actual,TRAD,Pago del día
1,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual,FP,Pago del día
2,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual,FP,Reprogramación
3,99659,d48beed5-51e0-43d7-84c8-57754b61f4a2,SUPERTICK,KQ32L6S5Z,2022-10-28,127.49,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,d48beed5-51e0-43d7-84c8-57754b61f4a2,2022-11-01,1,Actual,FP,Pago del día
4,99657,5665e03f-407f-49a8-bf85-ff4e28bf862d,City Cell San Nicolas,24T7JZ4J9,2022-10-28,400.62,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,5665e03f-407f-49a8-bf85-ff4e28bf862d,2022-11-01,1,Actual,FP,Pago del día


In [44]:
cruce_2.shape

(12172, 19)

In [45]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,101103,ee8c6855-c796-4db9-81f7-8534f353aa5b,Takos De kanasta Rios,LKHX3J5MR,2022-11-01,90.08,ERROR,R8,2022-11-01,11,2022,NEW,ERROR_USUAL,ee8c6855-c796-4db9-81f7-8534f353aa5b,2022-11-01,1,Actual,TRAD,Pago del día
1,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual,FP,Pago del día
2,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual,FP,Reprogramación
3,99659,d48beed5-51e0-43d7-84c8-57754b61f4a2,SUPERTICK,KQ32L6S5Z,2022-10-28,127.49,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,d48beed5-51e0-43d7-84c8-57754b61f4a2,2022-11-01,1,Actual,FP,Pago del día
4,99657,5665e03f-407f-49a8-bf85-ff4e28bf862d,City Cell San Nicolas,24T7JZ4J9,2022-10-28,400.62,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,5665e03f-407f-49a8-bf85-ff4e28bf862d,2022-11-01,1,Actual,FP,Pago del día


In [46]:
cruce_2_final.shape

(7259, 19)

In [47]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
1,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual,FP,Pago del día
2,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual,FP,Reprogramación
20,99384,7175596b-a83b-4e72-acd2-d389d58b0114,Barber,G62VWJWZ4,2022-10-28,287.47,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,7175596b-a83b-4e72-acd2-d389d58b0114,2022-11-01,1,Actual,FP,Pago del día
21,99384,7175596b-a83b-4e72-acd2-d389d58b0114,Barber,G62VWJWZ4,2022-10-28,287.47,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,7175596b-a83b-4e72-acd2-d389d58b0114,2022-11-01,1,Actual,FP,Reprogramación
22,99377,48374596-32ad-4e03-9604-344b9c952d62,Abarrotes Paty,CQB9QSVKT,2022-10-28,178.24,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,48374596-32ad-4e03-9604-344b9c952d62,2022-11-01,1,Actual,FP,Pago del día


In [48]:
duplicates_cruce_2_final.shape

(1454, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [49]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x               \
esquema_query                           ERROR_USUAL                
status_cubeta                                 ERROR                
tipo_dispersion                                  FP         TRAD   
tipo_pago                            Reprogramación Pago del día   
0                         2022-11-01       6,853.36         0.00   
1                         2022-11-02           0.00         0.00   
2                         2022-11-03      46,461.01         0.00   
3                         2022-11-04           0.00         0.00   
4                         2022-11-05           0.00         0.00   

                                                                         \
esquema_query                                                             
status_cubeta                           NEW                               
tipo_dispersion                          FP                        TRAD   
tipo_pago       Reprogramación Pago del día Reprogramación Pago del día   
0                    14,746.72     6,485.74       1,129.70    54,199.07   
1                         0.00         0.00           0.00    13,196.50   
2                   139,710.30     2,669.39           0.00   129,106.62   
3                     6,955.25         0.00           0.00   109,118.37   
4                         0.00         0.00           0.00     2,923.66   

                                           
esquema_query                         All  
status_cubeta                              
tipo_dispersion                            
tipo_pago       Reprogramación             
0                    32,802.08 116,216.67  
1                         0.00  13,196.50  
2                     3,273.34 321,220.66  
3                     2,605.45 118,679.07  
4                         0.00   2,923.66

In [50]:
group_tipo_pago.shape

(31, 9)

In [51]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only     amount_x                              \
esquema_query                       ERROR_USUAL                               
status_cubeta                             ERROR                         NEW   
tipo_pago                          Pago del día Reprogramación Pago del día   
0                       2022-11-01         0.00      21,600.08    60,684.81   
1                       2022-11-02         0.00           0.00    13,196.50   
2                       2022-11-03         0.00     186,171.31   131,776.01   
3                       2022-11-04         0.00       6,955.25   109,118.37   
4                       2022-11-05         0.00           0.00     2,923.66   

                                         
esquema_query                       All  
status_cubeta                            
tipo_pago     Reprogramación             
0                  33,931.78 116,216.67  
1                       0.00  13,196.50  
2                   3,273.34 321,220.66  
3                   2,605.45 118,679.07  
4                       0.00   2,923.66

In [52]:
group_esquema.shape

(31, 6)

# CRUCE 3 - MES ANTERIOR

In [53]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,21a9fde8-6e3c-4eca-9379-c364ccfbb71f,Helados Thrifty,jlpartida@hotmail.com,22236J8TB,2022-09-01,292.26,"Banco Santander, S.A.",2022-09-01 10:32:24,JOSE LUIS PARTIDA BOBADILLA,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,ed9cd493-d59c-4bc0-9897-1e2659c4b7ee,Dulces y mas el jordan,clanton14pecaslatozo@gmail.com,2228TT7PR,2022-09-01,565.37,"HSBC México, S.A.",2022-09-01 09:23:59,David Jordan Flores Villeda,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,9e649a79-8e0d-4827-a66f-28c2a9872a8b,Cremeria lety,letyvillanueva1118@gmail.com,222C6KQ6L,2022-09-01,125.52,"BBVA Bancomer, S.A.",2022-09-01 10:48:20,Marcos Castro Villanueva,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,INSTANT,c511de59-24c4-4e03-a547-73f35913a9d6,Farmacia VCR,tania.espinoza1@gmail.com,222DS8V5W,2022-09-01,57.49,"Opciones Empresariales del Noreste, S.A. de C....",2022-09-01 14:14:51,Tania Marlene Espinoza Garcia,...,FP,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,480ad868-c4b9-43ea-8eec-9972af6a5362,Be Lo Style,bgaliciag@hotmail.com,222R5XR8M,2022-09-01,"9,774.05","Banco Nacional de México, S.A.",2022-09-01 10:30:48,Berenice Galicia Garduño,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [54]:
cruce_3.shape

(8498160, 26)

In [55]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1976\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2571333,DAILY,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,esthibalizmartinezr@gmail.com,LNCNLVR9V,2022-08-11,0.01,"Banco Compartamos, S.A.",2022-09-29 18:00:58,Estela Rosario Rafael,...,TRAD,Reprogramación,0.01,2022-10-01,2022-10-01,10.00,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,ERROR_USUAL,NEW
2577824,DAILY,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,esthibalizmartinezr@gmail.com,P7X9DHYKR,2021-12-24,0.02,"Banco Compartamos, S.A.",2022-09-29 18:00:58,Estela Rosario Rafael,...,TRAD,Reprogramación,0.02,2022-10-01,2022-10-01,10.00,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,ERROR_USUAL,NEW
5092372,DAILY,7b3ed5f1-0fd0-44f8-9240-eec35a6d780f,Foto Cafe,fotocaffee@gmail.com,G9S4LHNHL,2022-10-25,364.14,BANAMEX,2022-10-26 15:26:52,Fernando Alberto Ortega,...,TRAD,Reprogramación,364.14,2022-10-27,2022-10-27,10.00,ERROR,DEV.SPEICUENTA INEXISTENTE,ERROR_USUAL,NEW
5145313,DAILY,c1ff0f5a-9a4c-48c7-8621-df7a9509c674,Abarrotes Castillo,albertobttes@gmail.com,ZXS9G8C5K,2022-10-25,89.11,"Banco Nacional de México, S.A.",2022-10-26 15:26:54,Refugio Alberto Bustamante Sandoval,...,TRAD,Reprogramación,89.11,2022-10-27,2022-10-27,10.00,ERROR,DEV.SPEICUENTA INEXISTENTE,ERROR_USUAL,NEW


In [56]:
cruce_3_final.shape

(4, 26)

In [57]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [58]:
duplicates_cruce_3_final.shape

(0, 26)

# CRUCE 4 - MES ANTERIOR

In [59]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
1,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
2,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
3,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
4,82019,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,LNCNLVR9V,2022-08-11,0.01,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación


In [60]:
cruce_4.shape

(12, 19)

In [61]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
1,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
2,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
3,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
4,82019,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,LNCNLVR9V,2022-08-11,0.01,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación


In [62]:
cruce_4_final.shape

(12, 19)

In [63]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
1,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
2,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
3,46519,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,P7X9DHYKR,2021-12-24,0.02,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación
4,82019,cd0f6233-aa3c-4914-b429-e56e68200452,Artesanias Esmar,LNCNLVR9V,2022-08-11,0.01,ERROR,DEV.SPEITIPO DE OPERACIoN ERRO,2022-10-01,10,2022,NEW,ERROR_USUAL,cd0f6233-aa3c-4914-b429-e56e68200452,2022-10-01,1,NaN,TRAD,Reprogramación


In [64]:
duplicates_cruce_4_final.shape

(12, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [65]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x       
esquema_query                           ERROR_USUAL    All
status_cubeta                                   NEW       
tipo_dispersion                                TRAD       
tipo_pago                            Reprogramación       
0                         2022-10-01           0.03   0.03
1                         2022-10-27         453.25 453.25
2                                All         453.28 453.28

In [66]:
group_tipo_pago_last_month.shape

(3, 3)

In [67]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only       amount                      
esquema_query                       ERROR_USUAL                   All
status_cubeta                               NEW                      
tipo_pago                          Pago del día Reprogramación       
0                       2022-10-01         0.00           0.12   0.12
1                       2022-10-27       453.25         453.25 906.50
2                              All       453.25         453.37 906.62

In [68]:
group_esquema_last_month.shape

(3, 4)

In [69]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL NOVIEMBRE 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()